In [13]:
import pandas as pd
df = pd.read_csv('../fake_job_postings.csv')


In [14]:
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
raw_text_cols = [col for col in df.columns if df[col].dtype == 'object' and col not in binary_cols + ['job_id']]

# print each count of unique value each in text columns
for col in raw_text_cols:
    print(f"Column: {col}, Unique values: {df[col].nunique()}")

# categorical columns (less than 150 unique values)
categorical_cols = [col for col in df.columns if 2 < df[col].nunique() < 150]
text_cols = [col for col in df.columns if df[col].dtype == 'object' and col not in categorical_cols + ['job_id']]

# print columns
print(f"Binary columns: {binary_cols}")
print(f"Categorical columns: {categorical_cols}")
print(f"Text columns: {text_cols}")

Column: title, Unique values: 11231
Column: location, Unique values: 3105
Column: department, Unique values: 1337
Column: salary_range, Unique values: 874
Column: company_profile, Unique values: 1709
Column: description, Unique values: 14801
Column: requirements, Unique values: 11967
Column: benefits, Unique values: 6204
Column: employment_type, Unique values: 5
Column: required_experience, Unique values: 7
Column: required_education, Unique values: 13
Column: industry, Unique values: 131
Column: function, Unique values: 37
Binary columns: ['telecommuting', 'has_company_logo', 'has_questions', 'fraudulent']
Categorical columns: ['employment_type', 'required_experience', 'required_education', 'industry', 'function']
Text columns: ['title', 'location', 'department', 'salary_range', 'company_profile', 'description', 'requirements', 'benefits']


In [15]:
# Check for null values
grouped_null_percentage = df.groupby('fraudulent').apply(lambda x: x.isnull().mean() * 100)
grouped_null_percentage = grouped_null_percentage.T.round(2)
grouped_null_percentage['Total'] = (df.isnull().mean() * 100).round(2)
grouped_null_percentage = grouped_null_percentage.sort_values(by='Total', ascending=False)
print(grouped_null_percentage)


fraudulent               0      1  Total
salary_range         84.45  74.25  83.96
department           64.75  61.32  64.58
required_education   44.99  52.08  45.33
benefits             40.25  42.03  40.34
required_experience  38.88  50.23  39.43
function             35.96  38.91  36.10
industry             27.20  31.76  27.42
employment_type      18.98  27.83  19.41
company_profile      15.99  67.78  18.50
requirements         14.94  17.78  15.08
location              1.92   2.19   1.94
description           0.00   0.12   0.01
job_id                0.00   0.00   0.00
telecommuting         0.00   0.00   0.00
has_questions         0.00   0.00   0.00
has_company_logo      0.00   0.00   0.00
title                 0.00   0.00   0.00
fraudulent            0.00   0.00   0.00


In [16]:
# Deduplicate based on all columns except 'job_id'
df_dedup = df.drop_duplicates(subset=[col for col in df.columns if col != 'job_id'])
print(f"Original shape: {df.shape}, Deduplicated shape: {df_dedup.shape}")

# Check class balance before and after deduplication
for name, data in [('Before dedup', df), ('After dedup', df_dedup)]:
    print(f"\n=== {name} ===")
    counts = data['fraudulent'].value_counts()
    percentages = data['fraudulent'].value_counts(normalize=True) * 100
    print(pd.DataFrame({'count': counts, 'percentage': percentages.round(2)}))

# Check for duplicates in text columns after deduping
for col in text_cols:
    # Only consider non-null values for duplicates
    non_null_mask = df_dedup[col].notna()
    num_dupes = df_dedup[non_null_mask].duplicated(subset=[col]).sum()
    print(f"Column '{col}' has {num_dupes} duplicate non-null values after deduplication.")



Original shape: (17880, 18), Deduplicated shape: (17599, 18)

=== Before dedup ===
            count  percentage
fraudulent                   
0           17014       95.16
1             866        4.84

=== After dedup ===
            count  percentage
fraudulent                   
0           16743       95.14
1             856        4.86
Column 'title' has 6368 duplicate non-null values after deduplication.
Column 'location' has 14153 duplicate non-null values after deduplication.
Column 'department' has 4934 duplicate non-null values after deduplication.
Column 'salary_range' has 1953 duplicate non-null values after deduplication.
Column 'company_profile' has 12608 duplicate non-null values after deduplication.
Column 'description' has 2797 duplicate non-null values after deduplication.
Column 'requirements' has 2983 duplicate non-null values after deduplication.
Column 'benefits' has 4287 duplicate non-null values after deduplication.


In [17]:
# Columns to consider for identifying duplicates (ignore 'job_id' and 'fraudulent')
cols_to_check = df.columns.difference(['job_id', 'fraudulent'])

# Identify duplicates based on all columns except 'job_id' and 'fraudulent'
dup_mask = df.duplicated(subset=cols_to_check, keep=False)
df_duplicates = df[dup_mask].sort_values(by=list(cols_to_check))
print(f"Found {df_duplicates.shape[0]} duplicated rows (ignoring job_id and fraudulent).")

# Check if duplicated rows have consistent 'fraudulent' labels
inconsistent_labels = df.groupby(list(cols_to_check))['fraudulent'].nunique()
inconsistent_count = (inconsistent_labels > 1).sum()
print(f"Number of groups with inconsistent 'fraudulent' labels: {inconsistent_count}")


Found 527 duplicated rows (ignoring job_id and fraudulent).
Number of groups with inconsistent 'fraudulent' labels: 0


In [18]:
# EDA for location column
# Focus on non-null locations
non_null_locations = df['location'].dropna()
comma_counts = non_null_locations.str.count(',')
location_structure = comma_counts.value_counts().sort_index()
print(location_structure)

non_standard_locations = non_null_locations[comma_counts == 15]
print(non_standard_locations.sample(1, random_state=42))

# Drop null locations first
non_null_df = df.dropna(subset=['location']).copy()

# Count commas in each location
non_null_df['comma_count'] = non_null_df['location'].str.count(',')

# Count how many entries have 0, 1, 2, 3+ commas per class
location_structure_by_class = (
    non_null_df.groupby('fraudulent')['comma_count']
    .value_counts()
    .unstack(fill_value=0)
    .sort_index(axis=1)
)

print("=== Comma Count Distribution by Fraudulent Class ===")
print(location_structure_by_class)



location
0        94
2     17320
3       104
4         7
5         4
6         2
8         1
10        1
15        1
Name: count, dtype: int64
13181    US, CA, Los Angeles, California, San Francisco...
Name: location, dtype: object
=== Comma Count Distribution by Fraudulent Class ===
comma_count  0      2   3   4   5   6   8   10  15
fraudulent                                        
0            86  16492  95   7   4   2   0   0   1
1             8    828   9   0   0   0   1   1   0


Presense of duplicated rows ignoring job_id & target label, duplicated rows all have the same target label

## Check for duplication based on partial matching

### Unknown replacement for selected columns
NA values -> 'unknown'
Certain columns have other "NA" like values: eg. Not Applicable, NaN, Unspecified. They have all been replaced with 'unknown'

### Location seperation/cleaning
- Check if non-null locations are seperated by ','
- Split based on ','

### Interested Columns
only location, employment_type, title, description and requirements are used to find duplicates.

### Chunk Below
- Cleans columns ['required_experience', 'required_education', 'industry', 'function'] by replacing N/A values with str("unknown")
- Splits column ['location'] into 3 columns ['location_country', 'location_state', 'location_city']
- Handles employment type by:
    - Combining 'other' & 'unknown'
    - Combining 'temporary' & 'contract'
- Removes duplicates with enhanced restrictions on employment type & location

In [19]:
# Cleaning Employment Type for deduplication
df_cleaning = df_dedup.copy()

# Clean columns by replacing nulls and unspecified values with 'unknown'
cols_to_clean = ['required_experience', 'required_education', 'industry', 'function']
unspecified_values = ['Not Applicable','NaN','not applicable', 'Unspecified', 'Other','Others','none', 'na', 'n/a', '', ' ', None]

for col in cols_to_clean:
    df_cleaning[col] = df_cleaning[col].replace(unspecified_values, 'unknown')
    df_cleaning[col] = df_cleaning[col].fillna('unknown')

for col in df_cleaning.columns:
    df_cleaning[col] = df_cleaning[col].fillna('unknown')

# Adding columns for location cleaning: [country, state, city]
def clean_location(loc):
    if pd.isna(loc) or loc in unspecified_values:
        return ("unknown", "unknown", "unknown")
    parts = loc.split(',')
    parts = [part.strip() if part.strip() not in unspecified_values else "unknown" for part in parts]
    # Pad with "unknown" if we don't have all three parts
    while len(parts) < 3:
        parts.append("unknown")
    return (parts[0], parts[1], parts[2])

df_cleaning_loc = df_cleaning['location'].apply(clean_location)
df_cleaning['location_country'] = df_cleaning_loc.apply(lambda x: x[0])
df_cleaning['location_state'] = df_cleaning_loc.apply(lambda x: x[1])
df_cleaning['location_city'] = df_cleaning_loc.apply(lambda x: x[2])

def simplify_employment_type(x):
    if pd.isna(x):
        return 'unknown'
    x = x.strip().lower()
    if x in ['full-time', 'part-time']:
        return x  # keep these separate
    elif x in ['contract', 'temporary']:
        return 'non-permanent'
    elif x in ['other', 'unknown', '']:
        return 'unknown'
    else:
        return 'unknown'
df_cleaning['employment_type_clean'] = df_cleaning['employment_type'].apply(simplify_employment_type)

# Define a new function to compare locations accounting for unknowns
def compare_locations(row1, row2):
    # Compare countries first
    if row1['location_country'] is None or row2['location_country'] is None:
        return True
    if row1['location_country'] != row2['location_country']:
        return False
    # If countries match, compare states (unless either is unknown)
    if (row1['location_state'] != 'unknown' and 
        row2['location_state'] != 'unknown' and 
        row1['location_state'] != row2['location_state']):
        return False
    # If states match or either is unknown, compare cities (unless either is unknown)
    if (row1['location_city'] != 'unknown' and 
        row2['location_city'] != 'unknown' and 
        row1['location_city'] != row2['location_city']):
        return False
    return True

# Create a new comparison key function that uses the location comparison
def comparison_key(row):
    # Get location info with handling for unknown values
    location_info = (
        row['location_country'],
        row['location_state'] if row['location_state'] != 'unknown' else None,
        row['location_city'] if row['location_city'] != 'unknown' else None
    )
    
    # Get employment type, None if unknown
    emp = None if row['employment_type_clean'] == 'unknown' else row['employment_type_clean']
    
    # Return tuple with all comparison fields
    return (location_info, row['title'], row['description'], row['requirements'], emp)

df_cleaning['dedup_key'] = df_cleaning.apply(comparison_key, axis=1)
df_nodup = df_cleaning.drop_duplicates(subset=['dedup_key'])

In [20]:
# Binary check
binary_cols = [col for col in df.columns if df[col].nunique() == 2]
df_binary = df[binary_cols]
df_binary = df_binary.apply(lambda x: x.map({'Yes': 1, 'No': 0, True: 1, False: 0}) 
                            if x.dtypes == 'object' else x)
corr_matrix = df_binary.corr()
print(corr_matrix)

                  telecommuting  has_company_logo  has_questions  fraudulent
telecommuting          1.000000         -0.019836       0.020345    0.034523
has_company_logo      -0.019836          1.000000       0.233932   -0.261971
has_questions          0.020345          0.233932       1.000000   -0.091627
fraudulent             0.034523         -0.261971      -0.091627    1.000000
